In [6]:
from glob import glob

In [3]:
import json
# Get all the frames of a single video (all frames in given directory)
def get_all_frames(path):
    files = glob('{}/*.json'.format(path))
    files.sort()
    ret = []
    for file in files:
        with open(file) as json_file:
            data = json.load(json_file)
            
        # Fix weird json syntax in file
        data = data[list(data.keys())[0]]
        t = {}
        t['filename'] = '{}/{}'.format(path, data['filename'])
        t['regions'] = []
        t['centers'] = []
        
        for region in data['regions']:
            t['regions'].append({
                'x': region['shape_attributes']['x'],
                'y': region['shape_attributes']['y'],
                'width': region['shape_attributes']['width'],
                'height': region['shape_attributes']['height']
            })
            
            t['centers'].append({
                'x': region['shape_attributes']['x'] + region['shape_attributes']['width']/2,
                'y': region['shape_attributes']['y'] + region['shape_attributes']['height']/2,
            })
            
        ret.append(t)
    return ret

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.animation as animation
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from IPython.display import clear_output
from PIL import ImageDraw, Image

def get_img_with_centers(frame):
    img = Image.open(frame['filename'])
    draw = ImageDraw.Draw(img)

    for region in frame['centers']:
        r = 4
        leftUpPoint = (region['x']-r, region['y']-r)
        rightDownPoint = (region['x']+r, region['y']+r)
        twoPointList = [leftUpPoint, rightDownPoint]
        draw.ellipse(twoPointList, fill=(255,0,0,255))
        
    return img
    


def generate_video(frame_info):
    
    c_frames = len(frame_info)

    fig,ax = plt.subplots(1, figsize=(14,8))
    fig.tight_layout()
    plt.axis('off')

    pbar = tqdm(total=c_frames)
    pbar.set_description("Generating frames")
    imgs = []
    for frame in frame_info[0:c_frames]:
        img = get_img_with_centers(frame)

        imgs.append([ax.imshow(img)])
        pbar.update(1)
    pbar.close()

    pbar = tqdm(total=1)
    pbar.set_description("Stitching frames together")
    ani = animation.ArtistAnimation(fig, imgs, interval=40, blit=True)
    pbar.update(1)
    pbar.close()

    pbar = tqdm(total=1)
    pbar.set_description("Prepare for display")
    tag = ani.to_html5_video()
    pbar.update(1)
    pbar.close()
    plt.close()

    clear_output()
    display(HTML(tag))

In [82]:
# Dataset used: https://github.com/sweetyy83/Lstn_fdst_dataset
frame_info = get_all_frames('data/train_data/82')
generate_video(frame_info)